# osiris

![img](https://dm2301files.storage.live.com/y4mmRC1xelS6Y6MEqUnZ-k2vjpADHpo6UMZAaZWROunr9-Ml5FYDlZ6WMxCGedy7NDhwDpusZdF5E1oLR5Qn6momydHe7tYUOMwNeFeGW7pUWkBjGPSnZp2sacYWs9IKkose6xjhSySL_v2tbfItRI7T_Pw_Tayhaa2F_vrwW6ucyr6WPa6s9DWH_if9Y5Y3yAU?width=375&height=250&cropmode=none)


osiris is a Python data processing and analysis environment for data-based computational conflict forecasting using very large datasets and graph-based methods and models and visualization, powered by scalable graph databases.

You can use osiris to analyze causal chains and networks of confict and violence around the world from realtime-updated, [automatically-encoded political event data](https://parusanalytics.com/eventdata/papers.dir/Schrodt_Yonamine_NewDirectionsInText.pdf) from projects like GDELT. This notebook gives an overview of the osiris project, the [GDELT project](https://www.gdeltproject.org/) data that osiris uses, how to import political event data using osiris either from the GDELT file server or from Google BigQuery, how to visualize and analyze it using Python, and how to load it into a TigerGraph graph server instance to efficiently run graph-centric queries on it to retrieve vertex-edge event data that can then be further analyzed.

## Notebook Environment Setup

In [1]:
import os, sys
# Check if running inside Colab or Kaggle
IN_COLAB = 'COLAB_GPU' in os.environ
IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
IN_HOSTED_NB = IN_COLAB or IN_KAGGLE
os.environ['IN_HOSTED_NB'] = str(IN_HOSTED_NB)

OS_NAME = sys.platform.upper()
if OS_NAME in ['LINUX', 'DARWIN'] and IN_HOSTED_NB:
  import subprocess
  print('Installing osiris from GitHub...')
  print(subprocess.run('if [ -d "osiris" ]; then rm -Rf osiris; fi', text=True, shell=True, check=True, capture_output=True).stdout)
  print(subprocess.run('git clone https://github.com/allisterb/osiris --recurse-submodule', text=True, shell=True, check=True, capture_output=True).stdout)
  print(subprocess.run('cd osiris && ./install', text=True, shell=True, check=True, capture_output=True).stdout)

# If we're not in a hosted nb env assume we're running Jupyter from the osiris project directory root
OSIRIS_PATH = '..' if not IN_HOSTED_NB else 'osiris'

# Import the osiris code and set the runtime env. 
sys.path.append(os.path.join(OSIRIS_PATH, 'osiris'))
sys.path.append(os.path.join(OSIRIS_PATH, 'ext'))
from osiris_global import set_runtime_env
set_runtime_env(interactive_nb=True)

## GDELT Event Data

*From the  [GDELT project](https://www.gdeltproject.org/) website*:
>The GDELT Project is a realtime network diagram and database of global human society for open research.
![gf](https://www.gdeltproject.org/images/spinningglobe.gif)

>The GDELT Project is an initiative to construct a catalog of human societal-scale behavior and beliefs across all countries of the world, connecting every person, organization, location, count, theme, news source, and event across the planet into a single massive network that captures what's happening around the world, what its context is and who's involved, and how the world is feeling about it, every single day.

The GDELT [event data](http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf) contains hundreds of millions of automatically coded events extracted from news stories daily using NLU methods and models. Each event data row contains the following fields:
1. *Actors*: Humans or organizations or states which initiate and are the target of event actions. Actors may have geographic information but not temporal. An event references exactly 2 actors: Actor1 and Actor2.
2. *Actions*: Codes and other information which describe each event. Actions have both temporal and spatial attributes: an event time plus some geo information like latitude / longitude. Actors and actions naturally form graphs with directed edges connecting Actor1-->Action-->Actor2. An Actor-Action edge may contain attributes like the event time and a complementary reverse edge to make querying easier e.g. Actor1----event1_date---->Action1----event1_date--->Actor2----->event2_date----->Action2----event2_date---->Actor3 
3. *SourceURL*: a URL that locates the *story* from which the event data was extracted.

osiris can extract data directly from the GDELT file server. The advantage of this method is that you don't need to have any special credentials or server access (remember we're interested *open-source* indicators.). All the data is downloaded directly to your client machine or notebook environment.

### Importing GDELT data from file server

osiris uses *DataSource* classes to manage importing tabular data. 

In [2]:
# Import data directly from GDELT file server
from data.gdelt import DataSource
import pandas as pd
gdelt = DataSource()

In [3]:
# Get event data for a 1 week period
events = gdelt.import_data('events', 'Apr-14-2022', 'Apr-20-2022')

Importing GDELT events data for 7 day(s) from 04-14-2022 to 04-20-2022...


Import GDELT events data:   0%|          | 0/7 [00:00<?, ?day/s]

Importing GDELT events data for 7 day(s) from 04-14-2022 to 04-20-2022 completed in 72.01 s.


About a week's worth of event data in 2022 consists of about 700K events takes up about 340MB RAM.

In [4]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707186 entries, 0 to 125669
Data columns (total 62 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   GLOBALEVENTID          707186 non-null  int64  
 1   SQLDATE                707186 non-null  int64  
 2   MonthYear              707186 non-null  int64  
 3   Year                   707186 non-null  int64  
 4   FractionDate           707186 non-null  float64
 5   Actor1Code             640700 non-null  object 
 6   Actor1Name             640700 non-null  object 
 7   Actor1CountryCode      408112 non-null  object 
 8   Actor1KnownGroupCode   9610 non-null    object 
 9   Actor1EthnicCode       3423 non-null    object 
 10  Actor1Religion1Code    10452 non-null   object 
 11  Actor1Religion2Code    2561 non-null    object 
 12  Actor1Type1Code        296023 non-null  object 
 13  Actor1Type2Code        19713 non-null   object 
 14  Actor1Type3Code        495 non-null 

In [5]:
events

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,1039297304,20210414,202104,2021,2021.2849,NaN,NaN,NaN,NaN,NaN,...,3,"Houston, Texas, United States",US,USTX,TX201,29.763300,-95.3633,1380948,20220414010000,https://www.sentinelsource.com/news/local/keen...
1,1039297305,20210414,202104,2021,2021.2849,AFR,AFRICA,AFR,NaN,NaN,...,1,United States,US,US,NaN,39.828175,-98.5795,US,20220414010000,https://www.kcrw.com/news/shows/greater-la/art...
2,1039297306,20210414,202104,2021,2021.2849,JUD,SUPREME COURT,NaN,NaN,NaN,...,3,"Round Rock, Texas, United States",US,USTX,NaN,30.508300,-97.6789,1366966,20220414010000,https://www.engadget.com/the-new-york-public-l...
3,1039297307,20210414,202104,2021,2021.2849,JUD,SUPREME COURT,NaN,NaN,NaN,...,2,"Texas, United States",US,USTX,NaN,31.106000,-97.6475,TX,20220414010000,https://www.engadget.com/the-new-york-public-l...
4,1039297308,20210414,202104,2021,2021.2849,JUD,SUPREME COURT,NaN,NaN,NaN,...,2,"Texas, United States",US,USTX,NaN,31.106000,-97.6475,TX,20220414010000,https://www.engadget.com/the-new-york-public-l...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125665,1040383216,20220420,202204,2022,2022.3014,cre,CREE,NaN,NaN,cre,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.cjvr.com/2022/04/20/first-nations-...
125666,1040383217,20220420,202204,2022,2022.3014,cre,CREE,NaN,NaN,cre,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.cjvr.com/2022/04/20/first-nations-...
125667,1040383218,20220420,202204,2022,2022.3014,cre,CREE,NaN,NaN,cre,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.cjvr.com/2022/04/20/first-nations-...
125668,1040383219,20220420,202204,2022,2022.3014,telOPP,TELUGU,NaN,NaN,tel,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.deccanchronicle.com/nation/politic...


Event data is highly denormalized with many redundancies for ease of querying and coded using a hierachical coding system called [CAMEO](http://data.gdeltproject.org/documentation/CAMEO.Manual.1.1b3.pdf) - Conflict and Mediation Event Observations

In [6]:
events[['EventCode', 'CAMEOCodeDescription']]

,EventCode,CAMEOCodeDescription
0,080,"Yield, not specified below"
1,042,Make a visit
2,100,"Demand, not specified below"
3,100,"Demand, not specified below"
4,100,"Demand, not specified below"
...,...,...
125665,060,"Engage in material cooperation, not spec below"
125666,073,Provide humanitarian aid
125667,090,"Investigate, not specified below"
125668,043,Host a visit


We can query and filter event data directly using the Pandas dataframe

In [7]:
# Find all events that were geolocated in Ukraine
uka_events = events[(events.ActionGeo_CountryCode == 'UP')]
uka_events

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
116,1039297420,20220414,202204,2022,2022.2849,NaN,NaN,NaN,NaN,NaN,...,4,"Kyiv, Kyyiv, Misto, Ukraine",UP,UP12,28554,50.4333,30.5167,-1044367,20220414010000,https://www.japantimes.co.jp/news/2022/04/14/w...
125,1039297429,20220414,202204,2022,2022.2849,NaN,NaN,NaN,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220414010000,https://www.missioncityrecord.com/news/b-c-exp...
126,1039297430,20220414,202204,2022,2022.2849,NaN,NaN,NaN,NaN,NaN,...,4,"Chernihiv, Chernihivs'ka Oblast', Ukraine",UP,UP02,28554,51.5055,31.2849,-1037057,20220414010000,https://wild941.com/2022/04/13/you-can-charge-...
127,1039297431,20220414,202204,2022,2022.2849,NaN,NaN,NaN,NaN,NaN,...,4,"Lviv, L'vivs'ka Oblast', Ukraine",UP,UP15,28555,49.8407,24.0305,-1045268,20220414010000,https://wild941.com/2022/04/13/you-can-charge-...
128,1039297432,20220414,202204,2022,2022.2849,NaN,NaN,NaN,NaN,NaN,...,4,"Lviv, L'vivs'ka Oblast', Ukraine",UP,UP15,28555,49.8407,24.0305,-1045268,20220414010000,https://wild941.com/2022/04/13/you-can-charge-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125624,1040383175,20220420,202204,2022,2022.3014,USAGOV,THE WHITE HOUSE,USA,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420234500,https://www.hellenicshippingnews.com/u-s-crude...
125625,1040383176,20220420,202204,2022,2022.3014,USAGOV,JOE BIDEN,USA,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420234500,https://www.agassizharrisonobserver.com/news/t...
125648,1040383199,20220420,202204,2022,2022.3014,VAT,VATICAN,VAT,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420234500,http://www.icatholic.org/article/christs-resur...
125651,1040383202,20220420,202204,2022,2022.3014,VAT,VATICAN,VAT,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420234500,http://www.icatholic.org/article/christs-resur...


So about 50K of 700K events last week were coded as happening in Ukraine, not surprising given recent events. Many of those related to use of military force.

In [8]:
# CAMEO code 190 denotes 'use of military force'
uka_events[uka_events.EventCode.str.startswith('190')]

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
487,1039297791,20220414,202204,2022,2022.2849,FRA,FRENCH,FRA,NaN,NaN,...,4,"Kyiv, Kyyiv, Misto, Ukraine",UP,UP12,28554,50.4333,30.5167,-1044367,20220414010000,https://www.nytimes.com/2022/04/13/world/europ...
1905,1039301458,20220414,202204,2022,2022.2849,GOV,GOVERNOR,NaN,NaN,NaN,...,4,"Kharkiv, Kharkivs'ka Oblast', Ukraine",UP,UP07,25036,49.9808,36.2527,-1041320,20220414013000,http://www.koreatimes.co.kr/www/world/2022/04/...
2231,1039301784,20220414,202204,2022,2022.2849,RUS,RUSSIAN,RUS,NaN,NaN,...,5,"Kherson Oblast, Khersons'ka Oblast', Ukraine",UP,UP08,28550,46.5000,34.0000,-1041362,20220414013000,https://www.understandingwar.org/backgrounder/...
2250,1039301803,20220414,202204,2022,2022.2849,RUS,RUSSIAN,RUS,NaN,NaN,...,4,"Kherson, Khersons'ka Oblast', Ukraine",UP,UP08,28557,46.6558,32.6178,-1041356,20220414013000,https://www.understandingwar.org/backgrounder/...
2252,1039301805,20220414,202204,2022,2022.2849,RUS,RUSSIA,RUS,NaN,NaN,...,5,"Kherson Oblast, Khersons'ka Oblast', Ukraine",UP,UP08,28550,46.5000,34.0000,-1041362,20220414013000,https://www.understandingwar.org/backgrounder/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118792,1040373135,20220420,202204,2022,2022.3014,RUS,RUSSIAN,RUS,NaN,NaN,...,4,"Vadym, Khersons'ka Oblast', Ukraine",UP,UP08,28553,46.1827,33.5971,-1057325,20220420221500,https://www.news8000.com/i/elderly-in-ukraine-...
118793,1040373136,20220420,202204,2022,2022.3014,RUS,RUSSIAN,RUS,NaN,NaN,...,4,"Vadym, Khersons'ka Oblast', Ukraine",UP,UP08,28553,46.1827,33.5971,-1057325,20220420221500,https://www.news8000.com/i/elderly-in-ukraine-...
118874,1040373217,20220420,202204,2022,2022.3014,UKR,UKRAINE,UKR,NaN,NaN,...,4,"Chernihiv, Chernihivs'ka Oblast', Ukraine",UP,UP02,28554,51.5055,31.2849,-1037057,20220420221500,http://www.msn.com/en-us/news/world/a-bomb-sni...
118949,1040373292,20220420,202204,2022,2022.3014,USA,UNITED STATES,USA,NaN,NaN,...,4,"Kyiv, Kyyiv, Misto, Ukraine",UP,UP12,28554,50.4333,30.5167,-1044367,20220420221500,http://www.msn.com/en-us/news/world/as-a-new-u...


In [9]:
# Import Folium to plot these military force events on a map
import folium
folium.Map(
    location=[48., 31.], 
    tiles="Stamen Toner",
    zoom_start=6
)

In [10]:
uka_map = folium.Map(
    location=[48., 31.], 
    #tiles="Stamen Toner",
    zoom_start=6
)
uka_map
uka_events_sample = uka_events[uka_events.EventCode.str.startswith('190')].sample(n=100)
for r in uka_events_sample.itertuples():
    m = folium.Marker(location=[r.ActionGeo_Lat, r.ActionGeo_Long],
                      icon=folium.Icon(color="red", icon="fire", prefix="glyphicon"),
                      tooltip=str(r.Actor1CountryCode) + '->' + str(r.EventCode) + ' ' +  str(r.CAMEOCodeDescription) + '->' + str(r.Actor2CountryCode) +' on ' + str(r.SQLDATE)
                     )
    m.add_to(uka_map)
uka_map

### Shaping tabular data into graph vertices and edges

The GDELT data schema is 'flat' and designed for easy of tabular querying and grouping. To be able to do graph and network queries it needs to be shaped.

In [12]:
from data.etl import shape_events_actors_vertices
events_vertices, actor1_vertices, actor2_vertices = shape_events_actors_vertices(uka_events_sample)

Hashing Actor1 ID:   0%|          | 0/100 [00:00<?, ?row/s]

Hashing Actor2 ID:   0%|          | 0/100 [00:00<?, ?row/s]

Creating Action ADM Code:   0%|          | 0/100 [00:00<?, ?row/s]

We create unique IDs for actors that can be linked to actions. We hash individual actor fields together to create a unique ID for each actor and then drop all the other actor fields from event data.

In [13]:
events_vertices

,ID,Actor1ID,Actor2ID,Date,IsRoot,MonthYear,Year,FractionDate,EventCode,CAMEOCodeDescription,...,Geo_ADMCode,AvgTone,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
5178,1039638339,UU0edFAf83PsHmLT8YxjSuee/JU=,UK6Dj7YHjBWh4iLV9jZdvYlKQGo=,2022-04-16,False,202204,2022,2022.2904,190,"Use conventional military force, not specifie...",...,UP12 28554,-7.407407,4,Kyiv; Kyyiv; Misto; Ukraine,UP,50.4333,30.5167,-1044367,20220416000000,https://www.wclk.com/2022-04-15/russia-ukraine...
68962,1040117975,Y+Tm0OgbsbkiAV9/ZghzKYz2TPA=,diAsw7jZ4iPmweEwlriUjz6amXk=,2022-04-19,False,202204,2022,2022.2986,190,"Use conventional military force, not specifie...",...,UP17 28558,-2.081165,4,Malinovka; Odes'ka Oblast; Ukraine,UP,46.9426,30.3835,-1045710,20220419150000,https://www.veteranstoday.com/2022/04/19/russi...
35499,1039334765,1+VpnEK6JAOcDXUg+Wi6FLQuOHM=,YcaJTx/YJ4wGFm41moZ9iObuhBQ=,2022-04-14,False,202204,2022,2022.2849,190,"Use conventional military force, not specifie...",...,UP00 25090,-0.799083,4,Donbas; Ukraine (general); Ukraine,UP,48.5000,38.5000,-1038077,20220414064500,https://www.msn.com/en-us/news/world/new-800-m...
117247,1039462542,tlifxqsNyCzxIJnRwtQKuZToQQw=,Ig/XTfxvb9kaqbuTwOjWKNMhAFA=,2022-04-14,False,202204,2022,2022.2849,190,"Use conventional military force, not specifie...",...,UP14 25090,-3.939056,4,Luhansk; Luhans'ka Oblast'; Ukraine,UP,48.5670,39.3171,-1045160,20220414213000,https://www.cnn.com/europe/live-news/ukraine-r...
10511,1039764638,nFd4dPWdcPpTWxN0BsDzC5+j0ns=,Bx9LEjyi6eFaA8fpE2ph5ZCLxmk=,2022-04-17,False,202204,2022,2022.2932,190,"Use conventional military force, not specifie...",...,UP17 28558,-2.816901,4,Odessa; Odes'ka Oblast; Ukraine,UP,46.4639,30.7386,-1049092,20220417033000,https://www.publicradiotulsa.org/2022-04-16/wh...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92402,1040000031,02e5HHIoXSTN/L67kr0lG1g+/9I=,QnK+wLAM2uY0xcgqu0EhuHPNly0=,2022-04-18,True,202204,2022,2022.2959,190,"Use conventional military force, not specifie...",...,UP,-3.809524,1,Ukraine,UP,49.0000,32.0000,UP,20220418224500,https://www.9news.com.au/national/st-marys-che...
29957,1040049588,vveKXD+9ZErjem8krH9oZ87AkTM=,qJEmuDRpUVtiI4v+dUqydwRTaGY=,2022-04-19,True,202204,2022,2022.2986,190,"Use conventional military force, not specifie...",...,UP02 28544,-3.512397,4,Polissya; Chernihivs'ka Oblast'; Ukraine,UP,51.6807,32.4218,11405037,20220419063000,https://www.ukrinform.net/rubric-ato/3461264-w...
76958,1040127349,BIWQdr/nUWOOdDWlQGkBieygz9U=,TzDUyfR/lqodWVk8ks3USmv88VM=,2022-03-20,False,202203,2022,2022.2192,190,"Use conventional military force, not specifie...",...,UP00 25090,0.000000,4,Donbass; Ukraine (general); Ukraine,UP,48.5000,38.5000,-1038077,20220419160000,https://www.europesun.com/news/272495564/russi...
9526,1039870134,nFd4dPWdcPpTWxN0BsDzC5+j0ns=,7X0CCCwuZcktEKby2gqK97ueYRI=,2022-04-18,True,202204,2022,2022.2959,190,"Use conventional military force, not specifie...",...,UP07 25036,-9.644670,4,Kharkiv; Kharkivs'ka Oblast'; Ukraine,UP,49.9808,36.2527,-1041320,20220418033000,https://menafn.com/1104037191/Zelensky-18-peop...


The actor information is now stored as separate entities that can be linked to actions.

In [15]:
actor1_vertices

,ActorID,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor1Geo_Type,Actor1Geo_FullName,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID
5178,UU0edFAf83PsHmLT8YxjSuee/JU=,UKR,UKRAINE,UKR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Moskva, Moskva, Russia",RS,RS48,25106,55.7522,37.6156,-2960561
68962,Y+Tm0OgbsbkiAV9/ZghzKYz2TPA=,UKR,UKRAINE,UKR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Nikolaev, Mykolayivs'ka Oblast', Ukraine",UP,UP16,28557,46.9659,31.9974,-1047257
35499,1+VpnEK6JAOcDXUg+Wi6FLQuOHM=,UKR,UKRAINE,UKR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Donbas, Ukraine (general), Ukraine",UP,UP00,25090,48.5000,38.5000,-1038077
117247,tlifxqsNyCzxIJnRwtQKuZToQQw=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10511,nFd4dPWdcPpTWxN0BsDzC5+j0ns=,RUS,RUSSIAN,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Russia,RS,RS,NaN,60.0000,100.0000,RS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92402,02e5HHIoXSTN/L67kr0lG1g+/9I=,GOV,FIREFIGHTER,NaN,NaN,NaN,NaN,NaN,GOV,NaN,NaN,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP
29957,vveKXD+9ZErjem8krH9oZ87AkTM=,UKR,UKRAINIAN,UKR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Volyn, Vitsyebskaya Voblasts', Belarus",BO,BO07,40000,55.1680,27.5831,11439848
76958,BIWQdr/nUWOOdDWlQGkBieygz9U=,RUS,RUSSIA,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Kinzhal, Magadanskaya Oblast', Russia",RS,RS44,25092,62.1500,152.2000,-2928006
9526,nFd4dPWdcPpTWxN0BsDzC5+j0ns=,RUS,RUSSIAN,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Russia,RS,RS,NaN,60.0000,100.0000,RS


We can visualize this data using Graphistry.

In [ ]:
# Uncomment and run below if running inside Colab and you want to pull env variables from a file called vars.env on your GDrive
# !pip install colab-env --upgrade
# import colab_env

In [23]:
# Start using Graphistry, you need GRAPHISTRY_USER and GRAPHISTRY_PASS env variables
from graphistry import graphistry
graphistry.register(api=3, username=os.environ['GRAPHISTRY_USER'], password=os.environ['GRAPHISTRY_PASS'], protocol='https', server='hub.graphistry.com')

In [27]:
g = graphistry.bind(source="Actor1ID", destination="Actor2ID", edge_title="Date")
g.edges(events_vertices).plot()